In [2]:
import torch
import torch.nn as nn
import numpy as np
import autograd.numpy as np
from autograd import grad
import autograd.numpy.random as npr
from torch.autograd import Variable
import torch
import torch.optim as optim

from autograd.core import primitive
#import tensorflow as tf
import math as mat

from matplotlib import pyplot as plt
%matplotlib inline


In [9]:
import torch.nn as nn
# import wandb
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 1)

        self.tanh = nn.Tanh()
        # self.energy_out = nn.Sequential(
        #                     nn.Linear(24, 1),nn.ReLU()
        # )

    def forward(self, x):
        # x = self.flatten(x)
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc2(x)
        x = self.tanh(x)
        x = self.fc3(x)
        x = self.tanh(x)
        y = self.fc4(x)
       
        return y

In [4]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.utils.data as data_utils
import torch
n_epochs=100

In [116]:
def get_ip():
    x_ip = 2 * torch.rand((50, 1)) - 1
    #x_ip = torch.from_numpy(x.reshape(1, -1))
    #print(type(x_ip))
    #print(x_ip)
    train_x = data_utils.TensorDataset(x_ip)
    #print(type(train_x))
    #train_x.size()
    train_dataloader = data_utils.DataLoader(train_x, batch_size=50)
    #print(len(train_dataloader))
    return train_dataloader

In [133]:
def get_res_Loss(X, model, res_loss):
    X.requires_grad = True
    y = model(X)
    dy_dx = torch.autograd.grad(y, X, create_graph=True)[0]
    return res_loss(y, -2*X*dy_dx)
    #return ((y + 2*X*dy_dx)**2)/torch.Tensor(X.size()[0])

def get_bc_loss(X_0, model, bc_loss):
    y_0 = model(X_0)
    return bc_loss(y_0, torch.Tensor(1))
    #return (y_0 - torch.Tensor(1))**2

In [134]:
def train(x_loader, model, optimizer, t, res_loss, bc_loss):
    accumulated_res_loss = 0
    accumulated_bc_loss = 0
    accumulated_loss = 0
    grad = []
    #print(x_loader[0])
    for i, X_in in enumerate(x_loader):
        X = X_in[0]
        print(X)
        y = model(X)
        #print(X)
        #print(len(X[0]))
        optimizer.zero_grad()
        res_loss = get_res_Loss(X, model, res_loss)
        X.requires_grad = False
        bc_loss = get_bc_loss(torch.Tensor(0), model, bc_loss)
        net_loss = res_loss + bc_loss
        accumulated_res_loss += res_loss.item()
        accumulated_bc_loss += bc_loss.item()
        accumulated_loss += net_loss.item()
        net_loss.backward()
        optimizer.step()
        if t==50 or t==100:
            for name, param in model.named_parameters():
                if param.requires_grad:
                    grad.append(tuple(name, param.data, param.grad))
                    #print(name, param.data, param.grad)
        print("Residual Loss: ",res_loss)
        print("Boundary Loss: ",bc_loss)

In [135]:
model = NeuralNetwork()
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay=3e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                    factor=0.1, patience=30, threshold=0.0001, 
                                                    threshold_mode='rel', cooldown=0, min_lr=1e-15, eps=1e-08)
#reg_loss =  torch.nn.MSELoss(beta=0.3)
res_loss = torch.nn.MSELoss()
bc_loss = torch.nn.MSELoss()
#class_loss = torch.nn.CrossEntropyLoss()
#ener_loss = torch.nn.L1Loss()
loss_weight  = 0.5



for t in range(n_epochs):
    # wandb.log({"Epoch Number ": t})
    x_loader = get_ip()
    print(len(x_loader))
    print(f"Epoch {t+1}\n-------------------------------")
    train(x_loader, model, optimizer, t, res_loss, bc_loss)

NeuralNetwork(
  (fc1): Linear(in_features=1, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=1, bias=True)
  (tanh): Tanh()
)
<class 'torch.Tensor'>
<class 'torch.utils.data.dataset.TensorDataset'>
1
1
Epoch 1
-------------------------------
tensor([[-0.3812],
        [ 0.2399],
        [-0.9540],
        [-0.2627],
        [ 0.1546],
        [ 0.4265],
        [-0.7386],
        [ 0.6072],
        [-0.5136],
        [-0.4792],
        [ 0.0809],
        [ 0.1115],
        [ 0.7821],
        [ 0.4215],
        [ 0.2888],
        [-0.5216],
        [ 0.0520],
        [ 0.3619],
        [ 0.0352],
        [ 0.8024],
        [-0.0642],
        [ 0.7471],
        [-0.3829],
        [-0.6750],
        [ 0.5459],
        [ 0.0413],
        [-0.0799],
        [ 0.2635],
        [ 0.7998],
        [ 0.1640],
        [-0.9436],
        [-0.3589],
    

RuntimeError: grad can be implicitly created only for scalar outputs

In [68]:
import numpy as np

# Create a 1D numpy array
array_1d = np.array([1, 2, 3, 4, 5])

# Convert the array to a 2D column vector
array_2d = array_1d.reshape(-1, 1)

print(array_2d)
# Output:
# [[1]
#  [2]
#  [3]
#  [4]
#  [5]]


[[1]
 [2]
 [3]
 [4]
 [5]]


In [121]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Generate random input data with shape (50, 1)
X = torch.rand((50, 1))

# Generate random target data with shape (50, 1)
y = torch.rand((50, 1))

# Create a TensorDataset object from the input and target data
dataset = TensorDataset(X, y)

# Create a DataLoader object with the dataset and a batch size of 50
batch_size = 50
dataloader = DataLoader(dataset, batch_size=batch_size)

# Define a simple linear model


class LinearModel(torch.nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.linear = torch.nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)


# Create an instance of the linear model
model = LinearModel()

# Define the optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Train the model for 10 epochs
epochs = 10
for epoch in range(epochs):
    for batch_idx, batch in enumerate(dataloader):
        # Get the input and target data for the current batch
        X_batch, y_batch = batch

        # Zero out the gradients from the previous iteration
        optimizer.zero_grad()
        print(X_batch)

        # Forward pass
        y_pred = model(X_batch)

        # Compute the loss
        loss = criterion(y_pred, y_batch)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Print some information about the training progress
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch}, Batch {batch_idx}, Loss {loss.item()}")

# Make predictions on the input data
y_pred = model(X)

# Print the predictions
print("Predictions:")
#print(y_pred)


tensor([[0.3433],
        [0.8054],
        [0.4142],
        [0.9477],
        [0.3647],
        [0.7008],
        [0.8965],
        [0.9300],
        [0.9958],
        [0.1340],
        [0.3135],
        [0.9227],
        [0.5449],
        [0.1141],
        [0.5702],
        [0.4080],
        [0.1542],
        [0.9494],
        [0.3340],
        [0.0465],
        [0.3173],
        [0.5327],
        [0.4176],
        [0.4184],
        [0.8406],
        [0.0121],
        [0.1980],
        [0.7205],
        [0.2852],
        [0.4537],
        [0.2398],
        [0.9759],
        [0.9849],
        [0.9663],
        [0.4826],
        [0.7567],
        [0.2375],
        [0.3807],
        [0.2406],
        [0.5806],
        [0.5064],
        [0.9124],
        [0.1270],
        [0.6383],
        [0.4818],
        [0.2831],
        [0.8548],
        [0.5012],
        [0.2299],
        [0.3760]])
Epoch 0, Batch 0, Loss 0.15152686834335327
tensor([[0.3433],
        [0.8054],
        [0.4142],
  

In [148]:
import torch

# Create a toy neural network with a single input and output
model = torch.nn.Linear(1, 1)

# Generate random input data with shape (batch_size, 1)
batch_size = 50
X = torch.randn((batch_size, 1))

# Set X to require gradients
X.requires_grad = True

# Compute the output of the neural network
y = model(X)
print(y)
#print(X)

# Compute the gradient of y with respect to X
dy_dx = torch.autograd.grad(y[0], X[0], create_graph=True)

# Compute the derivative of the output with respect to the input
#d2y_dx2 = torch.autograd.grad(dy_dx, X)[0]

# Print the result
print(dy_dx)


tensor([[ 0.4115],
        [ 1.5635],
        [ 0.0667],
        [-1.1132],
        [-1.0782],
        [ 1.3676],
        [ 0.2273],
        [ 0.0687],
        [ 0.1842],
        [-0.1622],
        [-0.8458],
        [ 0.9342],
        [ 1.2204],
        [-0.9464],
        [-2.3768],
        [ 0.7282],
        [-0.4659],
        [-0.1449],
        [ 1.0596],
        [-0.4627],
        [ 0.0107],
        [ 0.2685],
        [ 0.6195],
        [ 0.8953],
        [ 0.4070],
        [ 0.6639],
        [ 0.5228],
        [-1.4160],
        [ 0.1051],
        [ 0.0090],
        [ 0.7763],
        [-0.2247],
        [ 0.7152],
        [ 0.5923],
        [ 1.2651],
        [-1.7205],
        [-1.3492],
        [-0.8178],
        [ 0.7622],
        [-0.1610],
        [ 0.8663],
        [ 1.0093],
        [ 0.6537],
        [-0.9483],
        [-0.4426],
        [-0.4773],
        [-0.9341],
        [ 1.1678],
        [-1.7881],
        [ 0.0415]], grad_fn=<AddmmBackward0>)


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.